# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cidreira,-30.1811,-50.2056,25.60,84,100,4.02,BR,1701967310
1,1,nova sintra,14.8667,-24.7167,21.17,76,100,6.73,CV,1701967310
2,2,udachny,66.4167,112.4000,-25.31,75,99,1.26,RU,1701967310
3,3,wailua homesteads,22.0669,-159.3780,20.44,83,20,3.09,US,1701967310
4,4,adamstown,-25.0660,-130.1015,22.56,78,40,4.88,PN,1701967310


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 500,
    frame_height = 400,
    size = "Humidity",
    color = "City",
    alpha = .6,
    scale = .7
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,46,bredasdorp,-34.5322,20.0403,23.12,51,0,4.01,ZA,1701967313
112,112,east london,-33.0153,27.9116,24.52,73,0,3.60,ZA,1701967319
257,257,jiwani,25.0500,61.7417,24.48,44,0,3.95,PK,1701967332
318,318,mount isa,-20.7333,139.5000,21.87,49,0,0.00,AU,1701967337
343,343,tamanrasset,22.7850,5.5228,23.95,9,0,3.60,DZ,1701967340
347,347,kruisfontein,-34.0033,24.7314,21.59,64,0,1.30,ZA,1701967341
439,439,illizi,26.4833,8.4667,21.89,27,0,3.48,DZ,1701967350
498,498,huatabampo,26.8333,-109.6333,25.40,20,0,0.76,MX,1701967288


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
46,bredasdorp,ZA,-34.5322,20.0403,51,
112,east london,ZA,-33.0153,27.9116,73,
257,jiwani,PK,25.0500,61.7417,44,
318,mount isa,AU,-20.7333,139.5000,49,
343,tamanrasset,DZ,22.7850,5.5228,9,
347,kruisfontein,ZA,-34.0033,24.7314,64,
439,illizi,DZ,26.4833,8.4667,27,
498,huatabampo,MX,26.8333,-109.6333,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 100000
params = {
    'categories':'accommodation.hotel',
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make an API request using the params dictionary and convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bredasdorp - nearest hotel: Victoria Hotel
east london - nearest hotel: No hotel found
jiwani - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
tamanrasset - nearest hotel: فندق الأمان
kruisfontein - nearest hotel: Savoy
illizi - nearest hotel: بوناقة
huatabampo - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
46,bredasdorp,ZA,-34.5322,20.0403,51,Victoria Hotel
112,east london,ZA,-33.0153,27.9116,73,No hotel found
257,jiwani,PK,25.0500,61.7417,44,No hotel found
318,mount isa,AU,-20.7333,139.5000,49,Ibis Styles
343,tamanrasset,DZ,22.7850,5.5228,9,فندق الأمان
347,kruisfontein,ZA,-34.0033,24.7314,64,Savoy
439,illizi,DZ,26.4833,8.4667,27,بوناقة
498,huatabampo,MX,26.8333,-109.6333,20,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 500,
    frame_height = 400,
    color = "City",
    alpha = .6,
    scale = .7,
    hover_cols = ["Hotel Name", "Country"] 
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)